In [24]:
from pathlib import Path
from pandas import Series, DataFrame
from tqdm import tqdm

import imageio
import plotly as plp
import plotly.graph_objs as plgo
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
import cv2
import os
import math
import torch
import torchaudio
import tensorflow as tf

In [2]:
proj_pth = '/work/r08922a13/generative_inpainting'
data_pth = op.join(proj_pth, 'examples/esc50/mag_origin_npy')
os.chdir(proj_pth)

In [ ]:
spec = np.ones([256, 256])
plt.figure()
plt.imshow(spec, 'gray')

In [ ]:
for i, filename in enumerate(sorted(Path(data_pth).glob('*.npy'))):
    print(i, filename)
    
    spec = np.load(filename)
    print(spec.shape)
    h, w, _ = spec.shape
    spec = spec.reshape((h, w))
    print(spec.shape)
    plt.figure()
    plt.imshow(spec, 'gray')
    break

In [ ]:
def l1_loss(x, y):
    error = abs(x - y)
    print(sum(error))
    print(sum(sum(error)))
    return sum(sum(error))

In [ ]:
def psnr(target, ref):
    diff = ref - target
    diff = diff.flatten('C')
    rmse = math.sqrt(np.mean(diff ** 2.))
    return 20 * math.log10(1.0 / rmse)

In [ ]:
def create_mask(length, mask_type='square'):
    # For sorted.
    if length < 10:
        output_pth = op.join(proj_pth, f'examples/esc50/mask_256/mask_{mask_type}_00{length}')
    elif length < 100:
        output_pth = op.join(proj_pth, f'examples/esc50/mask_256/mask_{mask_type}_0{length}')
    else:
        output_pth = op.join(proj_pth, f'examples/esc50/mask_256/mask_{mask_type}_{length}')
    
    if op.isfile(output_pth):
        return output_pth
    
    h, w = spec.shape
    mid_point = h // 2, w // 2
    # print(mid_point, length)
    
    mask = np.zeros((h, w, 3), np.uint8)
    if mask_type == 'square':
        t = mid_point[0] - (length // 2) + 1
        b = mid_point[0] + (length // 2)
        l = mid_point[1] - (length // 2) + 1
        r = mid_point[1] + (length // 2)
        # print(t, b, l, r)
        mask = cv2.rectangle(mask, (l, t), (r, b), (255, 255, 255), -1)
    elif mask_type == 'time':
        area = length * length
        if area < h * 1:
            constant = 11
            height = area // constant
            t = mid_point[0] - (height // 2) + 1
            b = mid_point[0] + (height // 2)
            l = mid_point[1] - (constant // 2) + 1
            r = mid_point[1] + (constant // 2)
        else:
            width = area // h
            t = 0 + 1
            b = h
            l = mid_point[1] - (width // 2) + 1
            r = mid_point[1] + (width // 2)
        # print(t, b, l, r)
        mask = cv2.rectangle(mask, (l, t), (r, b), (255, 255, 255), -1)

    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # print(gray[110][:])

    plt.figure()
    plt.imshow(mask, 'gray')
    # print(mask[0])

    # This way is for showing the visualized results then convert to 1-channel.
    mask = mask[:, :, np.newaxis]
    # print(square_mask.shape)

    np.save(output_pth, mask)
    return output_pth + '.npy'

In [ ]:
def create_mask_base_on_time(length, mask_type='square'):
    # For sorted.
    if length < 10:
        output_pth = op.join(proj_pth, f'examples/esc50/mask_256/mask_{mask_type}_00{length}')
    elif length < 100:
        output_pth = op.join(proj_pth, f'examples/esc50/mask_256/mask_{mask_type}_0{length}')
    else:
        output_pth = op.join(proj_pth, f'examples/esc50/mask_256/mask_{mask_type}_{length}')
    
    if op.isfile(output_pth):
        return output_pth
    
    h, w = spec.shape
    mid_point = h // 2, w // 2
    # print(mid_point, length)
    
    mask = np.zeros((h, w, 3), np.uint8)
    if mask_type == 'square':
        area = h * length
        side = math.floor(math.sqrt(area))
        t = mid_point[0] - (side // 2) + 1
        b = mid_point[0] + (side // 2)
        l = mid_point[1] - (side // 2) + 1
        r = mid_point[1] + (side // 2)
        # print(t, b, l, r)
        mask = cv2.rectangle(mask, (l, t), (r, b), (255, 255, 255), -1)
    elif mask_type == 'time':
        width = length
        t = 0 + 1
        b = h
        l = mid_point[1] - (width // 2) + 1
        r = mid_point[1] + (width // 2)
        # print(t, b, l, r)
        mask = cv2.rectangle(mask, (l, t), (r, b), (255, 255, 255), -1)

    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # print(gray[110][:])

    plt.figure()
    plt.imshow(mask, 'gray')
    # print(mask[0])

    # This way is for showing the visualized results then convert to 1-channel.
    mask = mask[:, :, np.newaxis]
    # print(square_mask.shape)

    np.save(output_pth, mask)
    return output_pth + '.npy'

In [ ]:
output_pth = op.join(proj_pth, 'examples/esc50/mag_origin_npy/loss_test_output')
loss_pth = op.join(output_pth, 'l1_loss')
checkpoint = op.join(proj_pth, 'logs/full_model_esc50_origin_npy_test')
square_l1_loss = []
# Set 10 as progress unix for mask length. From 10 -> 400.
for i in range(64):
    image = filename
    
    mask_type = 'square'
    mask_length = (i + 1) * 4
    mask = create_mask_base_on_time(mask_length, mask_type)
    
    #print(op.basename(filename))
    #output_filename = op.basename(filename).split('.')[0] + f'_{mask_type}_{i}' + '.' + op.basename(filename).split('.')[1]
    #output = op.join(output_pth, output_filename)
    #print(output)
    '''
    cmd = f'python /work/r08922a13/generative_inpainting/test.py --image {image} --mask {mask} --output {output} --checkpoint {checkpoint}'
    print(cmd)
    os.system(cmd)
    
    inpaint_spec = np.load(output)
    h, w, _ = inpaint_spec.shape
    inpaint_spec = inpaint_spec.reshape((h, w))

    square_l1_loss.append(l1_loss(spec, inpaint_spec))
    #break
    
loss_filename = op.join(loss_pth, f'l1_loss_{mask_type}.txt')
with open(loss_filename, 'w') as f:
    for index in square_l1_loss:
        f.write(str(index)+'\n')
    '''

In [ ]:
output = '/work/r08922a13/generative_inpainting/examples/esc50/mag_origin_npy/loss_test_output/1-80841-A-13_spec_square_22.npy'
inpaint_spec = np.load(output)
h, w, _ = inpaint_spec.shape
inpaint_spec = inpaint_spec.reshape((h, w))

In [ ]:
loss = psnr(spec, inpaint_spec)
print(loss)

In [ ]:
mask_type = 'square'
loss_filename = op.join(output_pth, f'l1_loss_{mask_type}.txt')
with open(loss_filename, 'r') as f:
    sq_loss_list = f.read().splitlines()
print(sq_loss_list)
square_l1 = [float(i) for i in sq_loss_list]

mask_type = 'time'
loss_filename = op.join(output_pth, f'l1_loss_{mask_type}.txt')
with open(loss_filename, 'r') as f:
    time_loss_list = f.read().splitlines()
print(time_loss_list)
time_l1 = [float(i) for i in time_loss_list]

In [23]:
loss_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss'
loss_trace_list = []
for filename in sorted(Path(loss_pth).glob('*.txt')):
    print(filename)
    sound_name = op.basename(filename).split('.')[0].split('_')[0]
    loss_type = op.basename(filename).split('.')[0].split('_')[2]
    mask_type = op.basename(filename).split('.')[0].split('_')[-1]
    print(sound_name, loss_type, mask_type)
    with open(filename, 'r') as f:
        loss_list = f.read().splitlines()
        loss_list_float = [float(i) for i in loss_list]
        loss_trace_list.append([loss_list_float, sound_name, loss_type, mask_type])

/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss/1-46353-A-49_2_l1_loss_square.txt
1-46353-A-49 l1 square
/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss/1-46353-A-49_2_l1_loss_time.txt
1-46353-A-49 l1 time
/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss/4-181628-A-17_3_l1_loss_square.txt
4-181628-A-17 l1 square
/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss/4-181628-A-17_3_l1_loss_time.txt
4-181628-A-17 l1 time
/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss/4-185575-C-20_2_l1_loss_square.txt
4-185575-C-20 l1 square
/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_pretrain_audioset/loss/4-185575-C-20_2_l1_loss_time.txt
4-185575-C-20 l1 time
/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15

In [ ]:
loss_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_time_mask_within_v2/loss'
within_loss_trace_list = []
for filename in sorted(Path(loss_pth).glob('*.txt')):
    print(filename)
    sound_name = op.basename(filename).split('.')[0].split('_')[0]
    loss_type = op.basename(filename).split('.')[0].split('_')[2]
    mask_type = op.basename(filename).split('.')[0].split('_')[-1]
    print(sound_name, loss_type, mask_type)
    with open(filename, 'r') as f:
        loss_list = f.read().splitlines()
        loss_list_float = [float(i) for i in loss_list]
        within_loss_trace_list.append([loss_list_float, sound_name, loss_type, mask_type])

In [ ]:
loss_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_time_mask_only/loss'
only_loss_trace_list = []
for filename in sorted(Path(loss_pth).glob('*.txt')):
    print(filename)
    sound_name = op.basename(filename).split('.')[0].split('_')[0]
    loss_type = op.basename(filename).split('.')[0].split('_')[2]
    mask_type = op.basename(filename).split('.')[0].split('_')[-1]
    print(sound_name, loss_type, mask_type)
    with open(filename, 'r') as f:
        loss_list = f.read().splitlines()
        loss_list_float = [float(i) for i in loss_list]
        only_loss_trace_list.append([loss_list_float, sound_name, loss_type, mask_type])

In [ ]:
loss_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_time_wider/loss'
wider_loss_trace_list = []
for filename in sorted(Path(loss_pth).glob('*.txt')):
    print(filename)
    sound_name = op.basename(filename).split('.')[0].split('_')[0]
    loss_type = op.basename(filename).split('.')[0].split('_')[2]
    mask_type = op.basename(filename).split('.')[0].split('_')[-1]
    print(sound_name, loss_type, mask_type)
    with open(filename, 'r') as f:
        loss_list = f.read().splitlines()
        loss_list_float = [float(i) for i in loss_list]
        wider_loss_trace_list.append([loss_list_float, sound_name, loss_type, mask_type])

In [ ]:
loss_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_time_wider_l115/loss'
wider_l115_loss_trace_list = []
for filename in sorted(Path(loss_pth).glob('*.txt')):
    print(filename)
    sound_name = op.basename(filename).split('.')[0].split('_')[0]
    loss_type = op.basename(filename).split('.')[0].split('_')[2]
    mask_type = op.basename(filename).split('.')[0].split('_')[-1]
    print(sound_name, loss_type, mask_type)
    with open(filename, 'r') as f:
        loss_list = f.read().splitlines()
        loss_list_float = [float(i) for i in loss_list]
        wider_l115_loss_trace_list.append([loss_list_float, sound_name, loss_type, mask_type])

In [ ]:
err_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256/loss_test_output/audio/loss'
err_trace_list = []
for filename in sorted(Path(err_pth).glob('*.txt')):
    print(filename)
    sound_name = op.basename(filename).split('.')[0].split('_')[0]
    mask_type = op.basename(filename).split('.')[0].split('_')[-2]
    error_type = op.basename(filename).split('.')[0].split('_')[-1]
    print(sound_name, error_type, mask_type)
    with open(filename, 'r') as f:
        err_list = f.read().splitlines()
        err_list_float = [float(i) for i in err_list]
        err_trace_list.append([err_list_float, sound_name, error_type, mask_type])

In [ ]:
area_list = []
for i in range(64):
    length = (i+1) * 4
    area_list.append(length)
print(area_list)
area_list = np.array(area_list)

In [ ]:
square_l1_np = np.array(square_l1)
square_l1_around = np.around(square_l1_np, decimals=1)
print(square_l1_around)
square_l1_1f = ["{:0.1f}".format(v) for v in square_l1_around]
print(square_l1_1f)

time_l1_np = np.array(time_l1)
time_l1_around = np.around(time_l1_np, decimals=1)
print(time_l1_around)
time_l1_1f = ["{:0.1f}".format(v) for v in time_l1_around]
print(time_l1_1f)

In [ ]:
# Compare between time & square
# loss_trace_list = err_trace_list # Enable if testing the error on waveform
set_num = 2
trace01, name01, loss01, mask01 = loss_trace_list[set_num * 2]
trace02, name02, loss02, mask02 = loss_trace_list[set_num * 2 + 1]
if loss01 == 'mean': loss01 = 'mean L1'
print(trace01, name01, loss01, mask01)
print(trace02, name02, loss02, mask02)
trace_1 = plgo.Scatter(x=area_list, y=trace01, mode='markers', name=mask01)
trace_2 = plgo.Scatter(x=area_list, y=trace02, mode='markers', name=mask02)
data = [trace_1, trace_2]
layout = plgo.Layout(
    xaxis={'title':f'length ({name01})', 'titlefont':{'size':15}},
    yaxis={'title':f'loss ({loss01})', 'titlefont':{'size':15}})
fig = plgo.Figure(data=data, layout=layout)
plp.offline.iplot(fig, filename=loss01)

In [24]:
# Compare two kinds of
set_num = 3
constant = 1 # time = 1, square = 0
trace01, name01, loss01, mask01 = loss_trace_list[set_num * 2 + constant]
trace02, name02, loss02, mask02 = only_loss_trace_list[set_num * 6 + constant]
if loss01 == 'mean': loss01 = 'mean L1'
print(trace01, name01, loss01, mask01)
print(trace02, name02, loss02, mask02)
trace_1 = plgo.Scatter(x=area_list, y=trace01, mode='markers', name='audioset')
trace_2 = plgo.Scatter(x=area_list, y=trace02, mode='markers', name='only')
data = [trace_1, trace_2]
layout = plgo.Layout(
    xaxis={'title':f'length ({name01})', 'titlefont':{'size':15}},
    yaxis={'title':f'loss ({loss01})', 'titlefont':{'size':15}})
fig = plgo.Figure(data=data, layout=layout)
plp.offline.iplot(fig, filename=loss01)

[32.96755361557007, 71.78622913360596, 97.02457332611084, 125.76364660263062, 171.9505853652954, 229.320631980896, 283.2178144454956, 316.22097730636597, 352.7515592575073, 398.8796706199646, 438.4269452095032, 473.5025358200073, 515.1484899520874, 561.5686545372009, 599.0086140632629, 625.0436959266663, 666.4740061759949, 708.1622982025146, 752.6306681632996, 780.8769869804382, 823.161458492279, 854.591404914856, 902.9407353401184, 928.6565337181091, 971.2803392410278, 1013.494390964508, 1089.3139414787292, 1129.7076029777527, 1184.7722826004028, 1209.9026350975037, 1272.5931458473206, 1300.4186506271362, 1329.7318739891052, 1387.6848492622375, 1470.249882698059, 1506.7257871627808, 1540.0930032730103, 1603.2265677452087, 1700.5833892822266, 1754.6904754638672, 1764.8558297157288, 1842.7491002082825, 1885.0699372291565, 1950.745602607727, 1948.0082292556763, 2045.1938834190369, 2033.3942193984985, 2099.990580558777, 2175.1927437782288, 2149.4149718284607, 2211.2493691444397, 2268.4639

In [ ]:
# Compare to five kinds of models
set_num = 10
constant = 1 # time = 1, square = 0
trace01, name01, loss01, mask01 = loss_trace_list[set_num * 2 + constant]
trace02, name02, loss02, mask02 = within_loss_trace_list[set_num * 2 + constant]
trace03, name03, loss03, mask03 = only_loss_trace_list[set_num * 2 + constant]
trace04, name04, loss04, mask04 = wider_loss_trace_list[set_num * 2 + constant]
trace05, name05, loss05, mask05 = wider_l115_loss_trace_list[set_num * 2 + constant]
if loss01 == 'mean': loss01 = 'mean L1'
print(trace01, name01, loss01, mask01)
print(trace02, name02, loss02, mask02)
print(trace03, name03, loss03, mask03)
print(trace04, name04, loss04, mask04)
print(trace05, name05, loss05, mask05)
trace_1 = plgo.Scatter(x=area_list, y=trace01, mode='markers', name='rect')
trace_2 = plgo.Scatter(x=area_list, y=trace02, mode='markers', name='within')
trace_3 = plgo.Scatter(x=area_list, y=trace03, mode='markers', name='time')
trace_4 = plgo.Scatter(x=area_list, y=trace04, mode='markers', name='wider')
trace_5 = plgo.Scatter(x=area_list, y=trace05, mode='markers', name='widerl1up')
data = [trace_1, trace_2, trace_3, trace_4, trace_5]
layout = plgo.Layout(
    xaxis={'title':f'length ({name01})', 'titlefont':{'size':15}},
    yaxis={'title':f'loss ({loss01})', 'titlefont':{'size':15}})
fig = plgo.Figure(data=data, layout=layout)
plp.offline.iplot(fig, filename=loss01)

In [ ]:
loss_trace_list = err_trace_list # Enable if testing the error on waveform
set_num = 10
trace01, name01, loss01, mask01 = loss_trace_list[set_num * 2]
trace02, name02, loss02, mask02 = loss_trace_list[set_num * 2 + 1]
if loss01 == 'mean': loss01 = 'mean L1'
print(trace01, name01, loss01, mask01)
print(trace02, name02, loss02, mask02)
trace_1 = plgo.Scatter(x=area_list, y=trace01, mode='markers', name=mask01)
trace_2 = plgo.Scatter(x=area_list, y=trace02, mode='markers', name=mask02)
data = [trace_1, trace_2]
layout = plgo.Layout(
    xaxis={'title':f'length ({name01})', 'titlefont':{'size':15}},
    yaxis={'title':f'loss ({loss01})', 'titlefont':{'size':15}})
fig = plgo.Figure(data=data, layout=layout)
plp.offline.iplot(fig, filename=loss01)

In [ ]:
trace_0 = plgo.Scatter(x=area_list, y=square_l1_1f, mode='markers', name='square mask')
trace_1 = plgo.Scatter(x=area_list, y=time_l1_1f, mode='markers', name='time mask')
data = [trace_0, trace_1]
layout = plgo.Layout(
    xaxis={'title':'length', 'titlefont':{'size':20}},
    yaxis={'title':'loss (l1)', 'titlefont':{'size':20}})
fig = plgo.Figure(data=data, layout=layout)
plp.offline.iplot(fig, filename='loss')

In [ ]:
data = np.stack((area_list, time_l1_1f))
print(data, data.shape)

plt.figure()
plt.ylabel('l1loss')
plt.xlabel('length')
plt.plot(area_list, time_l1_1f, 'k', linestyle='dashed', marker='o')

In [ ]:
data_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/')
png_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/loss_test_time_mask_only/audio/png')
shape = ['square', 'time']
for filename in sorted(Path(data_pth).glob('*.npy')):
    #print(filename)
    file = op.basename(filename).split('.')[0]
    for anyshape in shape:
        #print(file, anyshape)
        file_shape = file + '_' + anyshape
        print(file_shape)
        with imageio.get_writer(op.join(png_pth, f'{file_shape}.gif'), mode='I', duration=0.2) as writer:
            for png in tqdm(sorted(Path(png_pth).glob(f'{file_shape}*.png'))):
                # print(png)
                image = imageio.imread(png)
                #plt.figure()
                #plt.imshow(image, 'gray')
                writer.append_data(image)
'''
mask_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/loss_test_output/png')
with imageio.get_writer(op.join(mask_pth, '../mask_square.gif'), mode='I', duration=0.2) as writer:
    for png in sorted(Path(mask_pth).glob('mask_square*')):
        print(png)
        image = imageio.imread(png)
        #plt.figure()
        #plt.imshow(image, 'gray')
        writer.append_data(image)
'''

In [27]:
# diff -> png -> gif
data_pth = op.join(proj_pth, 'examples/audioset/mag_seg15')
output_pth = op.join(proj_pth, 'examples/audioset/mag_seg15/loss_pretrain_esc50')
diff_pth = op.join(output_pth, 'diff_output')
if op.isdir(diff_pth) is False:
    os.mkdir(diff_pth)
shape = ['square', 'time']
for filename in sorted(Path(data_pth).glob('*.npy')):
    origin = np.load(filename)
    file = op.basename(filename).split('.')[0]
    for anyshape in shape:
        file_shape = file + '_' + anyshape
        for output in tqdm(sorted(Path(output_pth).glob(f'{file_shape}*.npy'))):
            #print(output)
            inpainted = np.load(output)
            diff = abs(inpainted - origin)
            diff_output = op.join(diff_pth, op.basename(output))
            #print(diff_output)
            np.save(diff_output, diff)

png_pth = op.join(diff_pth, 'png')
if op.isdir(png_pth) is False:
    os.mkdir(png_pth)
for i, filename in enumerate(sorted(Path(output_pth).glob('*.npy'))):
    print(i, filename)
    
    spec = np.load(filename)
    h, w, _ = spec.shape
    spec = spec.reshape((h, w))
    
    png_filename = op.basename(filename).split('.')[0] + '.png'
    png_filename = op.join(png_pth, png_filename)
    
    # First save the spec to png with force try.
    plt.imsave(png_filename, spec, cmap='gray')


for filename in sorted(Path(data_pth).glob('*.npy')):
    #print(filename)
    file = op.basename(filename).split('.')[0]
    for anyshape in shape:
        #print(file, anyshape)
        file_shape = file + '_' + anyshape
        print(file_shape)
        with imageio.get_writer(op.join(png_pth, f'{file_shape}.gif'), mode='I', duration=0.2) as writer:
            for png in tqdm(sorted(Path(png_pth).glob(f'{file_shape}*.png'))):
                # print(png)
                image = imageio.imread(png)
                #plt.figure()
                #plt.imshow(image, 'gray')
                writer.append_data(image)

100%|██████████| 64/64 [00:00<00:00, 90.65it/s] 


0 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_000.npy
1 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_001.npy
2 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_002.npy
3 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_003.npy
4 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_004.npy
5 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_005.npy
6 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_006.npy
7 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_square_007.npy
8 /work/r08922a13/generative_inpainting/examples/audiose

78 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_014.npy
79 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_015.npy
80 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_016.npy
81 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_017.npy
82 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_018.npy
83 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_019.npy
84 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_020.npy
85 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped10357_3_time_021.npy
86 /work/r08922a13/generative_inpainting/examples/audioset/mag_s

165 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_037.npy
166 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_038.npy
167 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_039.npy
168 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_040.npy
169 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_041.npy
170 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_042.npy
171 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_043.npy
172 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_square_044.npy
173 /work/r08922a13/generative_inpainting/exampl

238 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_046.npy
239 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_047.npy
240 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_048.npy
241 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_049.npy
242 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_050.npy
243 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_051.npy
244 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_052.npy
245 /work/r08922a13/generative_inpainting/examples/audioset/mag_seg15/loss_pretrain_esc50/snipped3292_2_time_053.npy
246 /work/r08922a13/generative_inpainting/examples/audioset/mag_

...

 22%|██▏       | 14/64 [00:00<00:00, 132.80it/s]

snipped7266_2_square


 22%|██▏       | 14/64 [00:00<00:00, 132.83it/s]

snipped7266_2_time


100%|██████████| 64/64 [00:00<00:00, 131.84it/s]


In [ ]:
# diff -> png -> gif
data_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/')
output_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/loss_test_time_wider_l115')
png_pth = op.join(output_pth, 'png')
shape = ['square', 'time']
if op.isdir(png_pth) is False:
    os.mkdir(png_pth)
for i, filename in tqdm(enumerate(sorted(Path(output_pth).glob('*.npy')))):
    # print(i, filename)
    
    spec = np.load(filename)
    h, w, _ = spec.shape
    spec = spec.reshape((h, w))
    
    png_filename = op.basename(filename).split('.')[0] + '.png'
    png_filename = op.join(png_pth, png_filename)
    
    # First save the spec to png with force try.
    plt.imsave(png_filename, spec, cmap='gray')


for filename in sorted(Path(data_pth).glob('*.npy')):
    #print(filename)
    file = op.basename(filename).split('.')[0]
    for anyshape in shape:
        #print(file, anyshape)
        file_shape = file + '_' + anyshape
        # print(file_shape)
        with imageio.get_writer(op.join(png_pth, f'{file_shape}.gif'), mode='I', duration=0.2) as writer:
            for png in tqdm(sorted(Path(png_pth).glob(f'{file_shape}*.png'))):
                # print(png)
                image = imageio.imread(png)
                #plt.figure()
                #plt.imshow(image, 'gray')
                writer.append_data(image)

In [ ]:
data_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/')
wav_pth = op.join(proj_pth, 'examples/esc50/mag_seg15_256/loss_test_time_mask_only/audio')
output_pth = op.join(wav_pth, 'loss')
if op.isdir(output_pth) is False:
    os.mkdir(output_pth)
png_pth = op.join(wav_pth, 'png')
if op.isdir(png_pth) is False:
    os.mkdir(png_pth)
shape = ['square', 'time']
for filename in sorted(Path(data_pth).glob('*_res.wav')):
    #print(filename)
    origin_waveform, sr = torchaudio.load(filename, normalization=True)
    #print("Min: {}\nMax: {}\nMean: {}".format(origin_waveform.min(), origin_waveform.max(), origin_waveform.mean()))
    #plt.figure()
    #plt.plot(origin_waveform[0])
    file = op.basename(filename).split('.')[0].split('_origin')[0]
    for anyshape in shape:
        #print(file, anyshape)
        file_shape = file + '_' + anyshape
        #print(file_shape)
        error_list = []
        for output in tqdm(sorted(Path(wav_pth).glob(f'{file_shape}*.wav'))):
            #print(output)
            inpainted_waveform, sr = torchaudio.load(output, normalization=True)
            #print("Min: {}\nMax: {}\nMean: {}".format(inpainted_waveform.min(), inpainted_waveform.max(), inpainted_waveform.mean()))
            #plt.figure()
            #plt.plot(inpainted_waveform[0])
            #print(inpainted_waveform.shape, _, l)
            diff = torch.mean(abs(inpainted_waveform - origin_waveform))
            # err = ((origin_waveform-inpainted_waveform).abs() / origin_waveform.abs()).median()
            err_value = diff.item()
            error_list.append(err_value)
            # print(diff, err)
            
            wave_pack = np.concatenate((origin_waveform, inpainted_waveform), axis=0)
            # print(wave_pack.shape)
            
            plt.figure()
            plt.ylim(-0.5, 0.5)
            plt.plot(wave_pack[0], 'b', wave_pack[1], 'r')
            png_name = op.basename(output).split('.')[0] + '.png'
            png_name = op.join(png_pth, png_name)
            plt.savefig(png_name)
        error_filename = op.join(output_pth, f'{file_shape}_mean-l1.txt')
        with open(error_filename, 'w') as f:
            for index in error_list:
                f.write(str(index)+'\n')

In [ ]:
test_file_list = '/work/r08922a13/generative_inpainting/data/esc50/spec_15seg/validation_static_view.flist'
train_file_list = '/work/r08922a13/generative_inpainting/data/esc50/spec_15seg/train_shuffled.flist'

with open(test_file_list, 'r') as f:
    file_list = f.read().splitlines()
    '''
    for line in tqdm(f.readlines()):
        print(line)
        filename = op.basename(line)
        print(filename)
        break
    '''
dict_category_list = dict()
for filename in sorted(file_list):
    print(filename)
    file_basename = op.basename(filename)
    category = file_basename.split('.')[0].split('_')[0].split('-')[-1]
    print(category)
    if int(category) < 10:
        category = '0' + category
    if dict_category_list.get(category) is None:
        dict_category_list[category] = []
    dict_category_list[category].append(5)
    #spec = np.load(file_list[0])

In [ ]:
print(dict_category_list['19'])
print(len(dict_category_list['19']))
arr = np.array(dict_category_list['19'])
print(arr)
print(arr.mean())

In [ ]:
test_file = '/work/r08922a13/generative_inpainting/scripts/testFile.txt'
with open(test_file, 'w') as f:
    # f.write("#list_id min, max mean percentile\n")
    for a_list in sorted(dict_category_list):
        print(a_list, dict_category_list[a_list])
        f.write(f'{dict_category_list[a_list]}\n')

In [ ]:
print(origin_waveform, inpainted_waveform)
print(abs(origin_waveform - inpainted_waveform))
print(torch.mean(abs(origin_waveform - inpainted_waveform)))

In [ ]:
test_waveform = torch.tensor([[1., -1., .5, .5, .2, .1, 1.]])
test2_waveform = torch.tensor([[1.5, -1.5, 1.5, .5, .2, .1, 1.]])
print(test_waveform.shape, test2_waveform.shape)
print((test_waveform - test2_waveform).abs())
print(test_waveform.abs())
print((test_waveform - test2_waveform).abs() / test_waveform.abs())
print(((test_waveform - test2_waveform).abs() / test_waveform.abs()).median())
print(test_waveform.median())

In [ ]:
test_waveform = torch.tensor([[1., -1., .5, .5, .2, .1, 1.]])
test2_waveform = torch.tensor([[1.5, -1.5, 1.5, .5, .2, .1, 1.]])
print(test_waveform.shape, test2_waveform.shape)
print(abs(test_waveform - test2_waveform))
print(torch.mean(abs(test_waveform - test2_waveform)))

In [83]:
#test_file = '/work/r08922a13/generative_inpainting/scripts/testFile.txt'
loss_pth = '/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256_all/train_m96/loss'
loss_trace_list = []
for filename in sorted(Path(loss_pth).glob('*.txt')):
    print(filename)
    loss_type = op.basename(filename).split('.')[0].split('_')[1]
    mask_type = op.basename(filename).split('.')[0].split('_')[0]
    print(loss_type, mask_type)
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.lstrip('[').rstrip(']\n')
            f_line = [float(num) for num in line.split(', ')]
            #print(line)
            loss_trace_list.append(f_line)
        #loss_list = f.read().lstrip('[').rstrip(']').splitlines()
        #loss_list_float = [float(i) for i in loss_list]
        #loss_trace_list.append([loss_list_float, loss_type, mask_type])

/work/r08922a13/generative_inpainting/examples/esc50/mag_seg15_256_all/train_m96/loss/m96_mean_l1_loss.txt
mean m96


In [ ]:
loss_trace_list

In [66]:
N = 50
x_data = ['dog', 'rooster', 'pig', 'cow', 'frog',
          'cat', 'hen', 'insects', 'sheep', 'crow',
          'rain', 'sea_waves', 'crackling_fire', 'crickets', 'chirping_birds',
          'water_drops', 'wind', 'pouring_water', 'toilet_flush', 'thunderstorm',
          'crying_baby', 'sneezing', 'clapping', 'breathing', 'coughing',
          'footsteps', 'laughing', 'brushing_teeth', 'snoring', 'drinking_sipping',
          'door_wood_knock', 'mouse_click', 'keyboard_typing', 'door_wood_creaks', 'can_opening',
          'washing_machine', 'vacuum_cleaner', 'clock_alarm', 'clock_tick', 'glass_breaking',
          'helicopter', 'chainsaw', 'siren', 'car_horn', 'engine',
          'train', 'church_bells', 'airplane', 'fireworks', 'hand_saw']
x_data_id = [str(i) for i in range(N)]
print(x_data_id)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49']


In [84]:
y_data = loss_trace_list

colors = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

fig = plgo.Figure()
for xd, yd, cls in zip(x_data, y_data, colors):
    fig.add_trace(plgo.Box(
        y=yd,
        name=xd,
        boxpoints='outliers', # False, 'all', 'suspectedoutliers', 'outliers'
        jitter=0.5,
        whiskerwidth=0.2,
        fillcolor=cls,
        marker_size=2,
        line_width=1)
    )
fig.update_layout(
    title=f'Mean-l1 loss on Esc-50 train data({mask_type})',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()